<a href="https://colab.research.google.com/github/ChatNoirGIGN/TA2---ProgramacionConcurrente/blob/main/TA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go spin 1> /dev/null && go version && spin -V



go version go1.10.4 linux/amd64
Spin Version 6.4.6 -- 2 December 2016


**INICIO:**

**Algoritmo utilizado:** Algoritmo matemático que, simulando el comportamiento de las neuronas, tiene la habilidad de aprender. Lo que consigue aprender dicho algoritmo son los pesos (Weights) adecuados para que la salida sea la correcta. Y como al final está decidiendo entre dos valores, podemos decir que el perceptrón está clasificando en dos clases los datos.

**IMPLEMENTACIÓN:**

In [ ]:
%%writefile Perceptron.go

#El paquete "perceptrón" proporciona la capacidad de crear, entrenar y utilizar un clasificador lineal que toma un número de entradas y devuelve una sola salida.

package perceptron

import (
	"fmt"
	"math/rand"
)

type Perceptron struct {
	Weights   []float64
	Threshold float64
}


# La función New crea un nuevo Perceptrón con el número especificado de entradas.
func New(numInputs int) *Perceptron {
	p := new(Perceptron)
	p.Weights = initializeWeights(numInputs)
	p.Threshold = rand.Float64()
	return p
}

func initializeWeights(numInputs int) []float64 {
	w := make([]float64, numInputs)
	for i := 0; i < numInputs; i++ {
		w[i] = rand.Float64()
	}
	return w
}

# La función "actividad" calcula la actividad en el Perceptrón para una entrada dada. No calcula si el Perceptrón realmente dispara, ver "Activación" en su lugar.
func (p *Perceptron) Activity(input []int) float64 {
	if len(p.Weights) != len(input) {
		panic(fmt.Sprintf("Input length should be %d", len(p.Weights)))
	}

	var sum float64

	for i := 0; i < len(p.Weights); i++ {
		sum += p.Weights[i] * float64(input[i])
	}

	return sum
}

#La función "Activates" comprueba si el Perceptrón se activa para la entrada dada. Devuelve 0 o 1 dependiendo de la entrada y de cómo el Perceptrón. Ha sido entrenado.
func (p *Perceptron) Activates(input []int) int {
	if len(p.Weights) != len(input) {
		panic(fmt.Sprintf("Input length should be %d", len(p.Weights)))
	}

	if p.Activity(input) >= p.Threshold {
		return 1
	} else {
		return 0
	}
}

#La función "Train" toma una entrada dada y una salida esperada. Si el Perceptrón no se activa, ajusta el umbral de activación y la entrada pesos. 
func (p *Perceptron) Train(input []int, expectedOutput int) {
	if len(p.Weights) != len(input) {
		panic(fmt.Sprintf("Input length should be %d", len(p.Weights)))
	}

	calculatedOutput := p.Activates(input)

	p.Threshold = p.Threshold - (float64(expectedOutput) - float64(calculatedOutput))

	for i := 0; i < len(p.Weights); i++ {
		p.Weights[i] = p.Weights[i] + (float64(expectedOutput)-float64(calculatedOutput))*float64(input[i])
	}
}

Overwriting Perceptron.go


**PRUEBAS:**

In [ ]:
%%writefile Perceptron_Test.go

package perceptron_test

import (
"github.com/onsi/ginkgo"
"github.com/onsi/gomega"

"testing"
)

func TestPerceptron(t *testing.T) {
	RegisterFailHandler(Fail)
	RunSpecs(t, "Perceptron Suite")
}

Overwriting Perceptron_Test.go


In [ ]:
%%writefile Perceptron_Test_2.go

package perceptron_test

import (
	"github.com/onsi/ginkgo"
	"github.com/onsi/gomega"

	"math/rand"

	"github.com/heldtogether/perceptron"
)

var _ = Describe("Perceptron", func() {

#Sembrar el generador de números aleatorios para que todos los números sean deterministas
	BeforeEach(func() {
		rand.Seed(1)
	})

	It("generates a set of weights on initialization", func() {
		numInputs := 2

		p := perceptron.New(numInputs)

# "Random" Weights = [0.6046602879796196 0.9405090880450124]
		Expect(p.Weights).Should(HaveLen(numInputs))
		Expect(p.Weights).Should(BeEquivalentTo([]float64{0.6046602879796196, 0.9405090880450124}))
	})

	It("generates a random threshold on initialization", func() {

		p := perceptron.New(2)

# "Random" Threshold = 0.6645600532184904
		Expect(p.Threshold).Should(BeNumerically("==", 0.6645600532184904))
	})

	It("can calculate the activity given an input", func() {

		p := perceptron.New(2)
		a := p.Activity([]int{1, 0})

# Calculated activity = 0.6046602879796196
		Expect(a).Should(BeNumerically("==", 0.6046602879796196))
	})

	It("adjusts weights and threshold when wrong", func() {

		p := perceptron.New(2)
		p.Train([]int{1, 0}, 1)

		Expect(p.Threshold).Should(BeNumerically("==", -0.33543994678150957))
		Expect(p.Weights[0]).Should(BeNumerically("==", 1.6046602879796197))
		Expect(p.Weights[1]).Should(BeNumerically("==", 0.9405090880450124))
	})

	It("activates when activity is greater than the threshold", func() {
		p := perceptron.New(2)
		p.Threshold = 0

# Calculated activity = 0.6046602879796196 > 0 so activates
		Expect(p.Activates([]int{1, 0})).Should(BeNumerically("==", 1))
	})

	It("doesn't activates when activity is greater than the threshold", func() {
		p := perceptron.New(2)
		p.Threshold = 1

# Calculated activity = 0.6046602879796196 < 1 so doesn't activate
		Expect(p.Activates([]int{1, 0})).Should(BeNumerically("==", 0))
	})

})


Overwriting Perceptron_Test_2.go


In [ ]:
!go run Perceptron_Test.go < input.file

/bin/bash: input.file: No such file or directory
